# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 11:33AM,246147,21,626367,"NBTY Global, Inc.",Released
1,Sep 12 2022 11:24AM,246146,10,CLI00377.1,"CLINUVEL, Inc.",Released
2,Sep 12 2022 11:24AM,246146,10,CLI00378.1,"CLINUVEL, Inc.",Released
3,Sep 12 2022 11:22AM,246144,21,626338,"NBTY Global, Inc.",Released
4,Sep 12 2022 10:55AM,246143,10,CTF0011404,"Citieffe, Inc.",Released
5,Sep 12 2022 10:40AM,246142,16,EMERSA-8516645,"Emersa Waterbox, LLC",Executing
6,Sep 12 2022 10:25AM,246131,20,ACI-8217699-BO,"ACI Healthcare USA, Inc.",Released
7,Sep 12 2022 10:25AM,246131,20,8506334,"ACI Healthcare USA, Inc.",Released
8,Sep 12 2022 10:25AM,246131,20,8515428,"ACI Healthcare USA, Inc.",Released
9,Sep 12 2022 10:25AM,246131,20,8515429,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,246142,Executing,1
25,246143,Released,1
26,246144,Released,1
27,246146,Released,2
28,246147,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246142,NaN,1.0,NaN
246143,NaN,NaN,1.0
246144,NaN,NaN,1.0
246146,NaN,NaN,2.0
246147,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0.0,0.0,1.0
246096,0.0,0.0,1.0
246103,2.0,5.0,0.0
246106,0.0,0.0,1.0
246109,0.0,13.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0,0,1
246096,0,0,1
246103,2,5,0
246106,0,0,1
246109,0,13,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,0,0,1
1,246096,0,0,1
2,246103,2,5,0
3,246106,0,0,1
4,246109,0,13,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,,,1
1,246096,,,1
2,246103,2,5,
3,246106,,,1
4,246109,,13,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc."
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc."
3,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc."
4,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc."
5,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC"
6,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc."
41,Sep 12 2022 10:20AM,246138,10,Bio-PRF
45,Sep 12 2022 10:19AM,246137,10,Bio-PRF
54,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.
60,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",,,1
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc.",,,2
2,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",,,1
3,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",,,1
4,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",,1,
5,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",,6,29
6,Sep 12 2022 10:20AM,246138,10,Bio-PRF,,,4
7,Sep 12 2022 10:19AM,246137,10,Bio-PRF,,5,4
8,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.,,,6
9,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",1,,
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc.",2,,
2,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",1,,
3,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",1,,
4,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",,1,
5,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",29,6,
6,Sep 12 2022 10:20AM,246138,10,Bio-PRF,4,,
7,Sep 12 2022 10:19AM,246137,10,Bio-PRF,4,5,
8,Sep 12 2022 10:18AM,246135,10,Eywa Pharma Inc.,6,,
9,Sep 12 2022 9:58AM,246060,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",1,,
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc.",2,,
2,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",1,,
3,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",1,,
4,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc.",2.0,NaN,NaN
2,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Sep 12 2022 11:24AM,246146,10,"CLINUVEL, Inc.",2.0,0.0,0.0
2,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2707429,99.0,20.0,0.0
12,246113,2.0,0.0,0.0
15,738377,19.0,15.0,0.0
16,246142,0.0,1.0,0.0
19,246103,0.0,5.0,2.0
20,492237,30.0,6.0,0.0
21,984447,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2707429,99.0,20.0,0.0
1,12,246113,2.0,0.0,0.0
2,15,738377,19.0,15.0,0.0
3,16,246142,0.0,1.0,0.0
4,19,246103,0.0,5.0,2.0
5,20,492237,30.0,6.0,0.0
6,21,984447,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,20.0,0.0
1,12,2.0,0.0,0.0
2,15,19.0,15.0,0.0
3,16,0.0,1.0,0.0
4,19,0.0,5.0,2.0
5,20,30.0,6.0,0.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,12,Released,2.0
2,15,Released,19.0
3,16,Released,0.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,20.0,0.0,15.0,1.0,5.0,6.0,0.0
Released,99.0,2.0,19.0,0.0,0.0,30.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,20.0,0.0,15.0,1.0,5.0,6.0,0.0
2,Released,99.0,2.0,19.0,0.0,0.0,30.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,20.0,0.0,15.0,1.0,5.0,6.0,0.0
2,Released,99.0,2.0,19.0,0.0,0.0,30.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()